In [1]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'diamond-bucket-db' # Mention the created S3 bucket name here
print("Using bucket " + bucket)

Using bucket diamond-bucket-db


In [3]:
df= pd.read_csv("Diamonds Prices2022.csv")
df.head()


,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df=df[(df['x']!=0) & df['y']!=0 & (df['z']!=0)]
df.describe()
df1=df.copy()
df1

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74
53939,53940,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64
53940,53941,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53941,53942,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43


In [5]:
def detect_outliers(df1, columns):
    outliers = {}
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[column] = df1[(df1[column] < lower_bound) | (df1[column] > upper_bound)][column]
    return outliers

In [6]:
columns_to_check = ['x', 'y', 'z', 'depth', 'table']
outliers = detect_outliers(df1, columns_to_check)
outliers_count = sum(len(outlier_values) for outlier_values in outliers.values())
outliers_count



3235

In [7]:
outlier_indices = set()

In [8]:
for outlier_values in outliers.values():
    outlier_indices.update(outlier_values.index)


In [9]:
data= df1.drop(index=outlier_indices)
data=data.drop('Unnamed: 0',axis=1)



In [10]:
features = list(data.columns)
features

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z']

In [11]:
label = features.pop(-4)
label

'price'

In [12]:
from sklearn.preprocessing import LabelEncoder
le_cut = LabelEncoder()
le_clarity = LabelEncoder()
le_color = LabelEncoder()

# Fit and transform each column
data['cut'] = le_cut.fit_transform(data['cut'])
data['clarity'] = le_color.fit_transform(data['clarity'])
data['color'] = le_color.fit_transform(data['color'])

In [13]:
x = data[features]
y = data[label]

In [14]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,2,1,3,61.5,55.0,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,3.89,3.84,2.31
3,0.29,3,5,5,62.4,58.0,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,4.34,4.35,2.75
5,0.24,4,6,7,62.8,57.0,3.94,3.96,2.48
...,...,...,...,...,...,...,...,...,...
53938,0.86,3,4,3,61.0,58.0,6.15,6.12,3.74
53939,0.75,2,0,3,62.2,55.0,5.83,5.87,3.64
53940,0.71,3,1,2,60.5,55.0,5.79,5.74,3.49
53941,0.71,3,2,2,59.8,62.0,5.74,5.73,3.43


In [15]:
y

0         326
1         326
3         334
4         335
5         336
         ... 
53938    2757
53939    2757
53940    2756
53941    2756
53942    2757
Name: price, Length: 50971, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()

In [18]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [19]:
X_train_scale = scale.fit_transform(X_train)
X_test_scale = scale.transform(X_test)

In [20]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test
print(trainX.shape)
print(testX.shape)

(43325, 10)
(7646, 10)


In [21]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [22]:
# send data to S3. SageMaker will take training data from s3
sk_prefix = "diamond"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [23]:
%%writefile script.py


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd

# inference functions ---------------

def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":
        request_body = request_body.strip()
        try:
            df = pd.read_csv(StringIO(request_body), header=None)
            return df
        
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def predict_fn(input_data, model):
    if type(input_data) != str:
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-4)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestRegressor(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = r2_score(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)

Overwriting script.py


In [24]:
! python script.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \


[INFO] Extracting arguments
SKLearn Version:  1.5.2
Joblib Version:  1.4.2
[INFO] Reading data

Building training and testing datasets

Column order: 
['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'y', 'z', 'price']

Label column is:  x

Data Shape: 

---- SHAPE OF TRAINING DATA (85%) ----
(43325, 9)
(43325,)

---- SHAPE OF TESTING DATA (15%) ----
(7646, 9)
(7646,)

Training RandomForest Model.....

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tre

In [25]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="Custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [26]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

INFO:sagemaker:Creating training-job with name: Custom-sklearn-2024-12-04-05-19-04-679


2024-12-04 05:19:05 Starting - Starting the training job...
2024-12-04 05:19:30 Starting - Preparing the instances for training...
2024-12-04 05:19:56 Downloading - Downloading input data...
2024-12-04 05:20:12 Downloading - Downloading the training image..
2024-12-04 05:21:08 Training - Training image download completed. Training in progress.
2024-12-04 05:21:08 Uploading - Uploading generated training model2024-12-04 05:20:54,194 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-12-04 05:20:54,196 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-04 05:20:54,232 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-12-04 05:20:54,373 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-04 05:20:54,385 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-04 05:20:54,396 sagemaker-training-toolkit INFO 

In [27]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2024-12-04 05:21:20 Starting - Preparing the instances for training
2024-12-04 05:21:20 Downloading - Downloading the training image
2024-12-04 05:21:20 Training - Training image download completed. Training in progress.
2024-12-04 05:21:20 Uploading - Uploading generated training model
2024-12-04 05:21:20 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-south-1-418295706442/Custom-sklearn-2024-12-04-05-19-04-679/output/model.tar.gz


In [28]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [29]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)


EndpointName=Custom-sklearn-model-2024-12-04-05-21-57


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-12-04-05-21-57
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2024-12-04-05-21-57
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2024-12-04-05-21-57


------!

In [30]:
testX[features][0:2].values.tolist()

[[0.51, 3.0, 0.0, 5.0, 59.7, 58.0, 5.18, 5.24, 3.11],
 [0.7, 1.0, 4.0, 7.0, 60.5, 62.0, 5.66, 5.74, 3.45]]

In [31]:
print(predictor.predict(testX[features][0:2].values.tolist()))

Please use Content-Type = 'text/csv' and, send the request!!
